In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [14]:
import cv2
import sys
import math
import time
import random
import os
from datetime import datetime
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim
from sewar.full_ref import vifp

In [15]:
from iEBMO import bmoIE

In [16]:
pSize = [50]
maxIter = 1
# imageNameList = ['./data/kodim/kodim04.png']
# imageNameList = ['a0027.tiff', 'a0037.tiff', 'a0050.tiff']
# imageNameList = ['a0027.png', 'a0037.png', 'a0050.png']

# print(imageNameList)

imageNameList = ['./data/kodim/kodim01.png', './data/kodim/kodim02.png', './data/kodim/kodim03.png', './data/kodim/kodim04.png',
                 './data/kodim/kodim05.png', './data/kodim/kodim06.png', './data/kodim/kodim07.png', './data/kodim/kodim08.png', 
                 './data/kodim/kodim09.png', './data/kodim/kodim10.png','./data/kodim/kodim11.png', './data/kodim/kodim12.png', 
                 './data/kodim/kodim13.png', './data/kodim/kodim14.png', './data/kodim/kodim15.png', './data/kodim/kodim16.png', 
                 './data/kodim/kodim17.png', './data/kodim/kodim18.png', './data/kodim/kodim19.png', './data/kodim/kodim20.png', 
                 './data/kodim/kodim21.png', './data/kodim/kodim22.png', './data/kodim/kodim23.png', './data/kodim/kodim24.png', ]
imageNameList

['./data/kodim/kodim01.png',
 './data/kodim/kodim02.png',
 './data/kodim/kodim03.png',
 './data/kodim/kodim04.png',
 './data/kodim/kodim05.png',
 './data/kodim/kodim06.png',
 './data/kodim/kodim07.png',
 './data/kodim/kodim08.png',
 './data/kodim/kodim09.png',
 './data/kodim/kodim10.png',
 './data/kodim/kodim11.png',
 './data/kodim/kodim12.png',
 './data/kodim/kodim13.png',
 './data/kodim/kodim14.png',
 './data/kodim/kodim15.png',
 './data/kodim/kodim16.png',
 './data/kodim/kodim17.png',
 './data/kodim/kodim18.png',
 './data/kodim/kodim19.png',
 './data/kodim/kodim20.png',
 './data/kodim/kodim21.png',
 './data/kodim/kodim22.png',
 './data/kodim/kodim23.png',
 './data/kodim/kodim24.png']

In [17]:
import datetime
for i in range(len(imageNameList)):
    averagePsnr = 0
    averageSsim = 0
    averageVif = 0
    print(imageNameList[i])
    metrics = []
    for popSize in pSize:
        print(popSize)

        inputName = "GroundTruth/kodakDataset/"+imageNameList[i]
        truthName = imageNameList[i]

        trImg = cv2.imread(truthName, 0)
        # cv2.imshow('input', trImg)
        histr = cv2.calcHist([trImg], [0], None, [256], [0, 256])
        plt.plot(histr)
        # histogram of ground truth
        name_image_list = imageNameList[i].split('/')
        time = datetime.datetime.now()
        print(time)
        # plt.savefig(f"./histogramsDIBCO/ht_{str(time).split('.')[1]}{name_image_list[-1]}") # salvar histograma da imagem de entrada
        plt.clf()
        # cv2.imwrite(f"./truthDIBCO/t_{str(time).split('.')[1]}{name_image_list[-1]}", trImg) # salvar imagem de entrada

        bestImage = deepcopy(trImg)

        maxPsnr = 0
        maxSsim = 0
        maxVif = 0
        maxfit = -1000
        for iteration in range(10):
            outputImage, fitval = bmoIE(truthName, popSize, maxIter)

            truthImage = cv2.imread(truthName, 0)
            psnrval = cv2.PSNR(outputImage, truthImage)
            ssimval = ssim(outputImage, truthImage)
            vifval = vifp(outputImage, truthImage)
            print(iteration,psnrval,ssimval,vifval,int((psnrval+ssimval+vifval)*100))
            print(psnrval, ssimval, vifval)
            
            if (psnrval+ssimval+vifval) > maxfit:
                maxfit = psnrval+ssimval+vifval
                maxPsnr = psnrval
                maxSsim = ssimval
                maxVif = vifval
                bestImage = deepcopy(outputImage)
            # cv2.imshow(str(iteration+1),bestImage)

        print(maxPsnr, maxSsim, maxVif)

        averagePsnr += maxPsnr
        averageSsim += maxSsim
        averageVif += maxVif
        print(i+1,averagePsnr,averageSsim,averageVif)
        
        ### AQUI VEM O CODIGO PARA SALVAR AS METRICAS EM ARQUIVO ###
        
        metric = {'pSize': (popSize + 1), 'psnr': maxPsnr, 'ssim': maxSsim, 'vif': maxVif}
        metrics.append(metric)
        
        # Gravar os dados no arquivo de texto
        with open(f'./metricas_txts/metricas_{name_image_list[-1]}.txt', 'w') as arquivo:
            for metrica in metrics:
                linha = f"pSize: {metrica['pSize']}, PSNR: {metrica['psnr']}, SSIM: {metrica['ssim']}, VIF: {metrica['vif']}\n"
                arquivo.write(linha)

        histr = cv2.calcHist([bestImage],[0],None,[256],[0,256])
        plt.plot(histr)
        time = datetime.datetime.now()
        # plt.savefig(f"./histogramsDIBCO/ho_{str(time).split('.')[1]}{name_image_list[-1]}") # salvar histograma da imagem de saida
        plt.clf()
        transImageName = "o_"+name_image_list[-1]
        # cv2.imwrite(f"./outputDIBCO/{str(time).split('.')[1]}{transImageName}", bestImage) # salvar imagem de saida


    averagePsnr /= len(imageNameList)
    averageSsim /= len(imageNameList)
    averageVif /= len(imageNameList)
    print("final")
    print(averagePsnr)
    print(averageSsim)
    print(averageVif)

./data/kodim/kodim01.png
50
2023-05-25 16:54:06.629025
254
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 

KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>